In [6]:
!pip install python-dotenv
!pip install openai

# Getting data from Mergeflow's API

In [11]:
import requests
import json
import os
from dotenv import load_dotenv # stores API keys for OpenAI and Mergeflow 

load_dotenv()  # Load variables from .env file
mergeflow_api_key = os.getenv('MERGEFLOW_API_KEY') # Assign Mergeflow API

# the API call
query = 'home appliance' # topic is factory automation
dataset = '&q=*%5bfundingcompany%7cfundingproject%7cfundingorganization%5d&sp=2572' # access dataset from mergeflow - in this case venture capital investments
rows = str(10) # first 10 docs only

# in the url below, replace "fw" by your user name. You can see your user name at mergeflow.net
# -> Account -> the string in angle brackets
url = 'https://mergeflow.net/api/v1/fw/search/getContent?q=' + query + dataset + '&rows=' + rows

# Define the headers to be sent with the request
headers = {
    'MergeflowNet-Api-Auth-Key': mergeflow_api_key
}

# Get and print the response
response = requests.post(url, headers=headers)

mergeflow_docs = []

# Check the response status code and content
if response.status_code == 200:
    mergeflow_response_json = response.json()
    
    # Pretty-print the JSON response -- useful to understand what the JSON looks like
    #pretty_json = json.dumps(mergeflow_response_json, indent=4)
    #print(pretty_json)
    
    # get title, URL, date, and content
    for document in mergeflow_response_json['Documents']:
        current_doc = {
            'title': document['Title'],
            'date': document['Date'],
            'url': document['Url'],
            'content': document['Content']
        }
        
        mergeflow_docs.append(current_doc)
        #print(content)        

    print('Found ' + str(len(mergeflow_docs)) + ' documents.')

else:
    print('Request failed with status code:', response.status_code)
    print(response.text)  # Print the response text


Found 10 documents.


In [12]:
import openai
import os
import requests
import json

openai.api_key = os.getenv('OPENAI_API_KEY')

summary_list = []

system_prompt_summary = f"""
You are a highly qualified academic scholar who has much experience in communicating your results to a broad audience. You are able to extract the most important aspects of texts, compress them and rephrase them in a comprehensible way.
"""

for doc in mergeflow_docs:
    user_prompt_summary = f"""
    Consider the text below, delimited by ```, in the area of {query}:
    ```
    {doc['content']}
    ```
    Please summarize the finding of the text in at most 500 words, and clearly cite the source at the end.
    """
    completion_summary = openai.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=[
            {"role": "system", "content": system_prompt_summary},
            {"role": "user", "content": user_prompt_summary}
        ],
        temperature=0.7,
        max_tokens=2000,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )

    summary_list.append(completion_summary.choices[0].message.content)
    print(completion_summary.choices[0].message.content)
    print("\n---\n")


print(summary_list)
print("\n----------------------------------------------\n----------------------------------------------\n")


Aliste Technologies recently secured $1 million in funding, with YourNest Venture Capital and Artha Venture Fund leading the investment round. Other participants included Dholakia Ventures, KRS Jamwal, and Anikarth Ventures, alongside existing investor 100X.VC. The company, established in 2021 by VIT batchmates Aakarsh Nayyar, Anant Ohri, Bhavya Kansal, Konark Gautam, Shreyansh Jain, and Udit Pandoh, focuses on the home automation market. Their goal is to make tech-enabled homes accessible to both homeowners and businesses seeking to enhance customer experiences and conserve energy.

Aliste Technologies offers retrofit automation products that empower users to control their current appliances through an app or voice assistants. Their subscription model starts at ₹1 per appliance per day. The company has already made an impact, with a presence in over 2,500 residential, commercial, and hospitality projects. Notable clients include The Taj group of hotels, Vouchagram India, and Stanza Li

Sepura Home, a Victoria, BC firm, has developed a revolutionary home appliance that transforms a kitchen sink into a composter. This innovative device, which replaces a traditional garbage disposal unit, is equipped with a Bluetooth button that can be conveniently placed near the sink for easy access. The system is installed underneath the sink and seamlessly connects to the drainage system.

Unlike conventional garbage disposals that grind food waste, Sepura's appliance operates by pushing the accumulated food scraps into the device when prompted by pressing the Bluetooth button. This approach aims to preserve the integrity of the food waste, as the company believes that keeping the waste intact is more beneficial for the environment by avoiding unnecessary crushing of materials into drains.

Moreover, the appliance incorporates safety features to prevent water from entering the receptacle until the drainage process is complete. It also has mechanisms in place to halt operations if it

UnityLab, a company based in Dushore, Pennsylvania, recently secured $4.5 million in Series A funding. The investment was supported by various entities, including Advantage Capital, the Bernstein Pennsylvania Fund, and the Ben Franklin Fund. Leveraging incentives from the Pennsylvania Rural Jobs and Investment Tax Credit Program, UnityLab plans to use the funds to expand its operations, create up to 150 jobs, and grow its headquarters and manufacturing facility in Dushore.

Led by CEO Jonathan Benjamin, UnityLab specializes in building and acquiring brands tailored to small businesses and addressing unmet customer needs. The company is known for its brands Hoffman-NewYorker and Unity Laundry Systems. Hoffman-NewYorker, a manufacturer of specialized presses, was formed through the merger of the Hoffman Pressing Machine Company and the NewYorker Pressing Machine Company. They offer garment and apparel presses for manufacturers, laundry and dry cleaning presses for retail and commercial u

In [13]:
Problem = """Based on the provided information, the problem with the highest overall rating is:
    - Problem: Inefficient and environmentally harmful management of food waste in households
    - User Group Impacted: homeowners and individuals seeking sustainable living practices.
    - Specific Pain Point: Lack of convenient and effective solutions for composting food waste at home.
    - Summary of Selection Criteria: Large target group impacted, very relevant pain point to home appliances and users, specific and critical issue in the industry."""
print(Problem)

Based on the provided information, the problem with the highest overall rating is:
    - Problem: Inefficient and environmentally harmful management of food waste in households
    - User Group Impacted: homeowners and individuals seeking sustainable living practices.
    - Specific Pain Point: Lack of convenient and effective solutions for composting food waste at home.
    - Summary of Selection Criteria: Large target group impacted, very relevant pain point to home appliances and users, specific and critical issue in the industry.


# Answering questions on the results with OpenAI GPT

In [14]:
import openai

openai.api_key = os.getenv('OPENAI_API_KEY') # Assign OpenAI API

#System prompt (for all users)
system_prompt = f"""You are Senior Vice President Technology and Innovation & CTO of Factory Automation. When you evaluate a new business idea, you strictly adhere to the following guiding principles:

        {Problem}
        
        - You are focusing more in SIEMENS, it would be great to emphasize about SIEMENS's strength
        - Focus on Sense&Act.
        - Be creative but grounded
        - Avoid homogeneous ideas
        """

summary_solutions = []

user_prompt = f"""
    Consider the text below, delimited by ```, in the area of {query}:
    ```
    {doc['content']}
    ```

    I wish to generate a business model canvas based on my finding and current trend, be detail as possible.
    """
    
completion = openai.chat.completions.create(
        model="gpt-4-0125-preview",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.8,
        max_tokens=2000,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )


print(completion.choices[0].message.content) # This prints the content of the output in a more readable way
print("\n---\n")  # This adds a separator between the messages for readability
summary_solutions.append(completion.choices[0].message.content)

To create a business model canvas for a venture addressing the inefficient and environmentally harmful management of food waste in households, leveraging Siemens's strength in technology and innovation, particularly in the Factory Automation sector, let's structure it according to the nine building blocks of the Business Model Canvas. This model will focus on the integration of smart technology for sustainable waste management solutions, emphasizing Siemens's "Sense&Act" capabilities.

### 1. Key Partnerships
- **Siemens's R&D and Technology Divisions**: For cutting-edge technology and innovation support.
- **Environmental Organizations**: To ensure sustainability practices and gain endorsements.
- **Local Governments**: For regulatory compliance and potential subsidies or incentives for promoting sustainability.
- **Waste Management Companies**: For potential collaboration in composting and recycling initiatives.
- **Retail Partners**: To distribute the product through various channel

In [15]:
# Initial rating of the solutions

initial_rating = []

system_prompt_2 = f"""You are a highly qualified critical rater of business opportunities. You are now tasked to evaluate a set of opportunities with the following problem in mind: {Problem} """


user_prompt4 = f""""

First, repeat the following points for the first solution as found in the following text: {summary_solutions}

    - "Name"
    - “Solved Problem”
    - “Solution”
    - “Impact”
    - “Technology”
    - “Sources”

Then, rate the first solution according to the following criteria. Please justify your ratings in max one sentence. Please be very critical in your ratings and only give a 3 out of 3 rating when it is very good:   

    - Functionality and Performance: Assess whether the solution addresses and solves the problem it’s supposed to solve. This includes estimating if the solution can solve the problem quickly and effectively. (1 = low performance, 2 = medium performance, 3 = high performance)
    - Scalability: Evaluate whether the solution can adapt to future growth (or shrinking) of demand and accommodate changes in requirements: How flexible is the solution? (1 = low scalability, 2 = medium scalability, 3 = high scalability)
    - Cost-effectiveness: Assess the costs associated with the solution. How large are upfront costs for implementation such as training, facilities, machines, etc.? How large are operating costs such as personnel, energy, maintenance, etc.? The solution should have a relatively high return on investment. (1 = low cost-effectiveness, 2 = medium cost-effectiveness 3 = high cost-effectiveness)
    - Innovativeness: How innovative is the solution? Do similar solutions for the problem exist already? (1 = low innovativeness, 2 = medium innovativeness 3 = high innovativeness)
    - Uniqueness: The solutions contained in the list should differ substantially and not be similar. (1 = low uniqueness, 2 = medium uniqueness 3 = high uniqueness)
    - Overall Rating: Sum of all the previous ratings, out of 15

Now, repeat these steps for all solutions.

"""


completion = openai.chat.completions.create(
model="gpt-4-0125-preview",
messages=[
{"role": "system", "content": system_prompt_2},
{"role": "user", "content": user_prompt4}
],
temperature=0.7,
max_tokens=2000,
top_p=0.95,
frequency_penalty=0,
presence_penalty=0,
stop=None
)

print(completion.choices[0].message.content) # This prints the content of the output in a more readable way
print("\n---\n")  # This adds a separator between the messages for readability
initial_rating.append(completion.choices[0].message.content)


### Siemens’ Smart Home Composting System

- **Name:** Siemens’ Smart Home Composting System
- **Solved Problem:** Inefficient and environmentally harmful management of food waste in households
- **Solution:** A smart, automated home composting system leveraging IoT for efficient food waste management.
- **Impact:** Reduces landfill contributions and methane emissions, provides valuable compost for gardening, and potentially reduces waste disposal fees.
- **Technology:** Siemens's "Sense&Act" capabilities for automated sensing of waste types and optimal composting action.
- **Sources:** Siemens’s R&D and Technology Divisions, Environmental Organizations, Local Governments, Waste Management Companies, Retail Partners.

#### Ratings:

- **Functionality and Performance:** 3
    - The solution directly addresses the problem with a focus on efficiency and effectiveness through automation and smart technology.
- **Scalability:** 2
    - While the solution seems adaptable to different home en

In [13]:
user_prompt_3 = f""""

Please take all the solutions and the corresponding ratings from: {initial_rating}

Take the three solutions with the highest absolute rating out of 15 (please be very diligent in this task. If there is an overall rating of 13, it should for example be outputed instead of an idea with a score of 10 out of 15). Sort the three in descending rating order.

Repeat the followinng points for each of the top three:

    - "Name"
    - “Solved Problem”
    - “Solution”
    - “Impact”
    - “Technology”
    - “Sources”
    - also include their rating from: {initial_rating} to double check they are in the top 3 highest rated


"""


completion = openai.chat.completions.create(
model="gpt-4-0125-preview",
messages=[
{"role": "system", "content": system_prompt_2},
{"role": "user", "content": user_prompt_3}
],
temperature=0.7,
max_tokens=2000,
top_p=0.95,
frequency_penalty=0,
presence_penalty=0,
stop=None
)

print(completion.choices[0].message.content) # This prints the content of the output in a more readable way
print("\n---\n")  # This adds a separator between the messages for readability

Based on the provided information, there is only one solution detailed, which is the Industrial-Control-as-a-Service (ICaaS). Since there are no other solutions provided to compare against, the ICaaS solution is by default the highest-rated option given, with an overall rating of 14 out of 15. Here are the details for the ICaaS solution:

- **Name:** Industrial-Control-as-a-Service (ICaaS)
- **Solved Problem:** Lack of seamless integration and communication between different factory automation systems and equipment.
- **Solution:** A roadmap for the development and growth of ICaaS, including cloud-based management of PLCs (Programmable Logic Controllers), Git-enabled PLC code versioning, virtualization of PLCs on edge servers, optimization based on feedback, expansion of virtual PLC functionality with AI for predictive maintenance, standardization across industries, and continuous innovation.
- **Impact:** Improved efficiency, reduced downtime, and enhanced performance through seamless